# Code to gather data from $z > 5.5$ QSOs.

We want to obtain data from several surveys and catalogs in order to  
manipulate them and try to obtain meaningful correlations.

First, we import the packages to be used

To get the first line working, you need  
to run the following lines:

```bash
 conda install nodejs
 pip install ipympl
 pip install --upgrade jupyterlab
 jupyter labextension install @jupyter-widgets/jupyterlab-manager
 jupyter labextension install jupyter-matplotlib
 jupyter nbextension enable --py widgetsnbextension
 ```

In [1]:
%matplotlib inline
# Static plots
#%matplotlib ipympl
# Interactive plots
import numpy as np
# import matplotlib.cm as cm
# import matplotlib.pyplot as plt
# import matplotlib.colors as mcolors
# import matplotlib.patheffects as mpe
# import matplotlib.patheffects as path_effects
# from matplotlib.ticker import FormatStrFormatter
# from matplotlib.patches import Polygon
from astropy.io import fits
from astropy.table import Table
from astropy.table import Column
from astropy.table import MaskedColumn
from astropy.table import hstack
from astropy.table import vstack
from astropy.table import join
from astropy.wcs import WCS
from astropy import units as u
# from astropy.visualization import hist
from astropy.coordinates import SkyCoord
from astroquery.simbad import Simbad
from astroquery.ned import Ned
import getpass
import progressbar

One function, to derive luminosity distances

In [2]:
def luminosity_distance(z_i, H0=70., WM=0.3, WV=0.7):
    z_i   = np.array([z_i], dtype='float64').flatten()
    c     = 299792.458 # velocity of light in km/sec
    h     = H0 / 100.
    WR    = 4.165E-5 / (h * h)   # includes 3 massless neutrino species, T0 = 2.72528
    WK    = 1 - WM - WR - WV
    azs   = 1.0 / (1 + z_i)
    DTT   = 0.0
    DCMR  = 0.0
    # do integral over a=1/(1+z) from az to 1 in n steps, midpoint rule
    n     = 1000  # number of points in integrals
    DL_Mpcs = np.zeros_like(z_i)
    for count, az in enumerate(azs):
        a     = az + (1 - az) * (np.arange(0, n) + 0.5) / n
        adot  = np.sqrt(WK + (WM / a) + (WR / (a * a)) + (WV * a * a))
        for i in range(n):
            # a    = az + (1 - az) * (i + 0.5) / n
            # adot = math.sqrt(WK + (WM / a) + (WR / (a * a)) + (WV * a * a))
            DTT  = DTT + 1. / adot[i]
            DCMR = DCMR + 1. / (a[i] * adot[i])
        DTT   = (1. - az) * DTT / n
        DCMR  = (1. - az) * DCMR / n
        # tangential comoving distance
        ratio = 1.00
        x     = np.sqrt(abs(WK)) * DCMR
        if x > 0.1:
            if WK > 0:
                ratio =  0.5 * (np.exp(x) - np.exp(-x)) / x 
            else:
                ratio = np.sin(x) / x
        else:
            y = x * x
        if WK < 0: y = -y
        ratio  = 1. + y / 6. + y * y / 120.
        DCMT   = ratio * DCMR
        DA     = az * DCMT
        DL     = DA / (az * az)
        DL_Mpc = (c / H0) * DL
        DL_Mpcs[count] = DL_Mpc
    return DL_Mpcs

Define the spectral index $\alpha$ from different sources  
to be used in the luminosity calculations (K-correction)

In [3]:
alpha_first = 0.5  # From FIRST data (Bornancini+2010)
alpha_RG    = 1.0  # For radio galaxies (Verkhodanov & Khabibullina, 2010)
alpha_alex  = 0.8  # Star-forming galaxies (Alexander+2003)
alpha_smol  = 0.7  # Mean value from VLA-COSMOS 3GHz sample (Smolčić et al. 2017)
alpha_butl  = 0.75  # From Butler et al., 2018

Choose one of the spectral indexes

In [4]:
alpha_used  = alpha_butl

---

### Reading data

Next step, reading our data.  
Most of the data files have been created using Topcat

In [5]:
machine  = getpass.getuser()
# cat_path = '/home/' + machine + '/Documentos/Data/'
cat_path = ''  # relative path to the same directory

Read data from FIRST+MILLIQUAS catalogs cross-matched.  

**MILLIQUAS + FIRST (SDSS)**   

Redshift values have been retrieved from the column `pipeline_redshift` in SDSS DR12  
The procedure to obtain these values is explained in **Bolton+2012**  

We also select sources with have explicitely data in both SDSS Quasar Catalog and  
in the FIRST survey (`first_match_flag = 1`).  

In order to get only the best redshift values, we select sources with  
`pipeline_redshift_flag = 0`.

Thus, we can use 9161 objects from the catalog.

In [6]:
#hdu_list = fits.open(cat_path + 'tables_matches_milli_sdss_apr.fits');
#sdss_milli  = Table(hdu_list[1].data);
#hdu_list.close();

sdss_milli = Table.read(cat_path + 'tables_matches_milli_sdss_apr.fits')

L_14GHz_filter = np.array((sdss_milli['L_14GHz'] > 0.0) * (sdss_milli['first_offset'] < 1.0)) ;  # sdss + milliquasar

We load data from the VLA-COSMOS 3 GHz Large Project
**Smolčić et al. 2017**

We can, also, separate sources which have an AGN X-ray counterpart
with the flag `cosm_xray_flag`

In [7]:
# hdu_cosmos          = fits.open(cat_path + 'vla_xmmcosmos_smolcic.fits')
# cosmos_data         = Table(hdu_cosmos[1].data)
# hdu_cosmos.close()

cosmos_data         = Table.read(cat_path + 'vla_xmmcosmos_smolcic.fits')

x_ray_flag          = cosmos_data['cosm_xray_flag']
redshift_cosmos     = cosmos_data['cosm_z']
redshift_cosmos[~np.isfinite(redshift_cosmos)] = 0.0  # Repair NaN values
int_14GHz_cosmos    = cosmos_data['cosm_lum_14ghz']  # Using their own spectral indices
cosmos_3GHz_flux    = cosmos_data['cosm_f_3ghz']  # mJy
cosmos_3GHz_flux_e  = cosmos_data['cosm_f_3ghz_err']
cosmos_3GHz_flux_e[~np.isfinite(cosmos_3GHz_flux_e)] = 0.0  # Repair NaN values
cosmos_14GHz_flux   = cosmos_3GHz_flux_e * (3/1.4)**alpha_used
cosmos_14GHz_flux[~np.isfinite(cosmos_14GHz_flux)] = 0.0  # Repair NaN values
cosmos_14GHz_flux_e = np.abs(cosmos_14GHz_flux) * cosmos_3GHz_flux_e / cosmos_3GHz_flux
lum_dist_cosmos     = cosmos_data['D_lum']  # in m
lum_dist_cosmos[~np.isfinite(lum_dist_cosmos)] = 0.0  # Repair NaN values

redshift_cosmos_x   = cosmos_data['cosm_z'][(cosmos_data['cosm_z'] > 0) * (cosmos_data['cosm_xray_flag'] == 'T')]
int_14GHz_cosmos_x  = cosmos_data['cosm_lum_14ghz'][(cosmos_data['cosm_z'] > 0) * (cosmos_data['cosm_xray_flag'] == 'T')]

We want, also, to add $z > 6$ QSOs from the list in  
Table 3 in the review of **Inayoshi, Visbal, and Haiman, 2020**.  
Six of them have $z > 7$

Not all of them have $1.4$ GHz measurements. Others have  
measurements in different frequencies which can be translated  
into the desired frequency using, for instance, the relation  
from **Butler et al., 2018**:

$$S_{a} = S_{b} \times (\frac{\nu_{b}}{\nu_{a}})^{\alpha}$$  

We load the data from these sources. Fluxes from different frequencies than $1.4$ GHz are translated to the needed value.

In [8]:
high_z_ra        = np.char.replace(np.loadtxt(cat_path + 'high_z_qso_props.csv', usecols=[1],  dtype='str', delimiter=';'), ',', '.').astype(np.float)
high_z_dec       = np.char.replace(np.loadtxt(cat_path + 'high_z_qso_props.csv', usecols=[2],  dtype='str', delimiter=';'), ',', '.').astype(np.float)
high_z_zs        = np.char.replace(np.loadtxt(cat_path + 'high_z_qso_props.csv', usecols=[3],  dtype='str', delimiter=';'), ',', '.').astype(np.float)
high_z_zs_e      = np.char.replace(np.loadtxt(cat_path + 'high_z_qso_props.csv', usecols=[4],  dtype='str', delimiter=';'), ',', '.').astype(np.float)
high_z_14GHz     = np.char.replace(np.loadtxt(cat_path + 'high_z_qso_props.csv', usecols=[6],  dtype='str', delimiter=';'), ',', '.').astype(np.float)
high_z_14GHz_e   = np.char.replace(np.loadtxt(cat_path + 'high_z_qso_props.csv', usecols=[7],  dtype='str', delimiter=';'), ',', '.').astype(np.float)
high_z_3GHz      = np.char.replace(np.loadtxt(cat_path + 'high_z_qso_props.csv', usecols=[8],  dtype='str', delimiter=';'), ',', '.').astype(np.float)
high_z_3GHz_e    = np.char.replace(np.loadtxt(cat_path + 'high_z_qso_props.csv', usecols=[9],  dtype='str', delimiter=';'), ',', '.').astype(np.float)
high_z_15GHz     = np.char.replace(np.loadtxt(cat_path + 'high_z_qso_props.csv', usecols=[10], dtype='str', delimiter=';'), ',', '.').astype(np.float)
high_z_15GHz_e   = np.char.replace(np.loadtxt(cat_path + 'high_z_qso_props.csv', usecols=[11], dtype='str', delimiter=';'), ',', '.').astype(np.float)
high_z_250GHz    = np.char.replace(np.loadtxt(cat_path + 'high_z_qso_props.csv', usecols=[12], dtype='str', delimiter=';'), ',', '.').astype(np.float)
high_z_250GHz_e  = np.char.replace(np.loadtxt(cat_path + 'high_z_qso_props.csv', usecols=[13], dtype='str', delimiter=';'), ',', '.').astype(np.float)
high_z_mass_1450 = np.char.replace(np.loadtxt(cat_path + 'high_z_qso_props.csv', usecols=[14], dtype='str', delimiter=';'), ',', '.').astype(np.float)
high_z_names     = np.loadtxt('high_z_qso_props.csv', usecols=[0], dtype='str', delimiter=';')
high_z_lum_d     = luminosity_distance(high_z_zs) * 3.086e22  # in m
high_z_up_lim    = np.array([val == '<' for val in np.loadtxt('high_z_qso_props.csv', usecols=[5], dtype='str', delimiter=';')])

Accumulate values into one array except 250GHz data.  
Millimetre luminosities will be used separately since we cannot be completely  
sure that they represent, fully, non-thermal emission (from AGN) and not dust.

In [9]:
high_z_14   = high_z_14GHz + high_z_3GHz + high_z_15GHz
high_z_14_e = high_z_14GHz_e + high_z_3GHz_e + high_z_15GHz_e

To complement the dataset, we also load four $z > 5.5$ sources which  
come from the `radio` catalog in the `Heasarc` database.  
We queried the objects which have $1.4$ GHz observations that  
are within a $2.5$ arcsec of an object from the `SDSS QUASAR DR12`  
catalog. We discard the sources that are already included in the `FIRST`  
catalog, to avoid repetition.

To enlarge the size of the sample, we can also query  
the same sample, but extending the redshift range to all positive  
values. We can include, too, the exclusion of ***low quality*** redshift_values (or not).

In [10]:
# hdu_radio_z_55       = fits.open(cat_path + 'radio_cat_sdss_z_55.fits');  # high redshift
# hdu_radio_z_all_spec = fits.open(cat_path + 'radio_cat_sdss_z_all_spec.fits');  # all redshift, high z quality
#hdu_radio_z_all_all  = fits.open(cat_path + 'radio_cat_sdss_z_all_all.fits');  # all redshift, all z quality
# radio_sources        = Table(hdu_radio_z_55[1].data);
# radio_sources        = Table(hdu_radio_z_all_spec[1].data);
# hdu_radio_z_55.close();
# hdu_radio_z_all_spec.close();
# hdu_radio_z_all_all.close();

# radio_sources        = Table.read(cat_path + 'radio_cat_sdss_z_55_spec.fits')       # high redshift
radio_sources        = Table.read(cat_path + 'radio_cat_sdss_z_all_spec.fits')      # all redshift, high z quality
# radio_sources        = Table.read(cat_path + 'radio_cat_sdss_z_all_all_spec.fits')  # all redshift, all z quality

---

### Calculate luminosities

Calculate luminosities (in W/Hz) for different datasets  
using the expression

$$L_{1.4\mathrm{GHz}} = 4 \pi \mathrm{d}^{2}_{L} f_{1.4\mathrm{GHz}} (1 + z)^{\alpha - 1}$$

which comes from Alexander et al. 2003

We can also obtain that luminosity from the flux in $3$ GHz as

$$L_{1.4\mathrm{GHz}} = 4 \pi \mathrm{d}^{2}_{L} {(\frac{3}{1.4})}^{\alpha} f_{3\mathrm{GHz}} (1 + z)^{\alpha - 1}$$

This expression comes from Delhaize et al. 2017.

In [11]:
L_21cm           = 4 * np.pi * (sdss_milli['D_lum'][L_14GHz_filter])**2 * sdss_milli['flux_20_cm'][L_14GHz_filter] * 1e-3 * 1e-26 * (1 + sdss_milli['redshift'][L_14GHz_filter])**(alpha_used - 1)
L_21cm_e         = np.abs(L_21cm) / sdss_milli['snr_20_cm'][L_14GHz_filter]

In [12]:
L_21cm_radio     = 4 * np.pi * (radio_sources['D_lum'])**2 * radio_sources['radio_flux'] * 1e-3 * 1e-26 * (1 + radio_sources['sdss_z'])**(alpha_used - 1)
L_21cm_radio_e   = np.abs(L_21cm_radio) * radio_sources['radio_flux_err'] / radio_sources['radio_flux']

In [13]:
L_14GHz_cosmos   = 4 * np.pi * (lum_dist_cosmos)**2 * cosmos_14GHz_flux * 1e-3 * 1e-26 * (1 + redshift_cosmos)**(alpha_used - 1)
L_14GHz_cosmos_e = np.abs(L_14GHz_cosmos) * cosmos_14GHz_flux_e / cosmos_14GHz_flux;
L_14GHz_cosmos_e[~np.isfinite(L_14GHz_cosmos_e)] = 0.0  # Repair (make zero) error values

/Users/rcarvajalp/.anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


In [14]:
high_z_lum_14    = 4 * np.pi * high_z_lum_d**2 * high_z_14GHz  * 1e-6 * 1e-26 * (1 + high_z_zs)**(alpha_used - 1)
high_z_lum_3     = 4 * np.pi * high_z_lum_d**2 * high_z_3GHz   * 1e-6 * 1e-26 * (1 + high_z_zs)**(alpha_used - 1) * (3/1.4)**alpha_used
high_z_lum_15    = 4 * np.pi * high_z_lum_d**2 * high_z_15GHz  * 1e-6 * 1e-26 * (1 + high_z_zs)**(alpha_used - 1) * (1.5/1.4)**alpha_used
high_z_lum_250   = 4 * np.pi * high_z_lum_d**2 * high_z_250GHz * 1e-6 * 1e-26 * (1 + high_z_zs)**(alpha_used - 1) * (250/1.4)**(alpha_used)

Mix all luminosities (different bands) to obtain single value (adding zeroes).  
Millimetre luminosities will be used separately since we cannot be completely  
sure that they represent, fully, non-thermal emission (from AGN) and not dust.

In [15]:
high_z_lum_14GHz   = high_z_lum_14 + high_z_lum_3 + high_z_lum_15

We can also determine error values for these luminosities

In [16]:
high_z_lum_14GHz_e  = np.zeros_like(high_z_lum_14GHz)
high_z_lum_250GHz_e = np.zeros_like(high_z_lum_250)
for counter, element in enumerate(high_z_lum_14GHz):
    if element == 0: continue
    high_z_lum_14GHz_e[counter] = np.abs(element) * high_z_14_e[counter] / high_z_14[counter]
for counter, element in enumerate(high_z_lum_250):
    if element == 0: continue
    high_z_lum_250GHz_e[counter] = np.abs(element) * high_z_250GHz_e[counter] / high_z_250GHz[counter]

Create a filter to plot, when needed, only the sources which  
have mm data but not radio observations.

In [17]:
filter_250GHz = np.array((high_z_lum_250 > 0) * (high_z_lum_14GHz == 0))

Now, we can use the points we are interested in. Our sample from **Inayoshi et al., 2020** and the  
sources from **SDSS+FIRST** with $z>5.5$.

Another option to display the data is, instead of showing redshift in the  
horizontal axis, have the mass of the observed objects.

**Inayoshi et al., 2020** use the rest-frame UV magnitude $\mathrm{M}_{1450}$  
to calculate the mass as:

$$M = 10^{[-(\mathrm{M}_{1450} + 3.459) / 2.5]} [\mathrm{M}_{\odot}]$$

which yields, on average, the published virial mass estimates for those available.

Create formal arrays from catalogs to merge them

Arrays from `SDSS+FIRST`

In [18]:
#limit_z_sdss          = 5.5
limit_z_sdss          = 0.0
filter_sdss_z         = np.array(sdss_milli['redshift'][L_14GHz_filter] > limit_z_sdss)

In [19]:
upper_sdss_L          = L_21cm[filter_sdss_z]
upper_sdss_L_e        = L_21cm_e[filter_sdss_z]
upper_sdss_u_lim      = np.zeros_like(upper_sdss_L, dtype=np.bool)
upper_sdss            = sdss_milli[np.array(sdss_milli['redshift'] > limit_z_sdss) * L_14GHz_filter]
upper_sdss_ra         = upper_sdss['sdss_ra']
upper_sdss_dec        = upper_sdss['sdss_dec']
upper_sdss_z          = upper_sdss['redshift']
upper_sdss_z_e        = upper_sdss['redshift_err']
upper_sdss_f_20cm     = upper_sdss['flux_20_cm']  # mJy
upper_sdss_f_20cm_e   = upper_sdss['flux_20_cm'] / upper_sdss['snr_20_cm']  # mJy
upper_sdss_f_250GHz   = np.zeros_like(upper_sdss_L)
upper_sdss_f_250GHz_e = np.zeros_like(upper_sdss_L)
upper_sdss_L_250GHz   = np.zeros_like(upper_sdss_L)
upper_sdss_L_250GHz_e = np.zeros_like(upper_sdss_L)
upper_sdss_mass_1450  = np.zeros_like(upper_sdss_L)

Arrays from the `COSMOS` Field

In [20]:
# upper_cosmos_L          = int_14GHz_cosmos[np.array(cosmos_data['cosm_z'] > limit_z_sdss)]  # Calculated by them
upper_cosmos_L          = L_14GHz_cosmos[np.array(redshift_cosmos > limit_z_sdss)]  # Calculated with our spectral index
upper_cosmos_L_e        = L_14GHz_cosmos_e[np.array(redshift_cosmos > limit_z_sdss)]
upper_cosmos_u_lim      = np.zeros_like(upper_cosmos_L, dtype=np.bool)
upper_cosmos            = cosmos_data[np.array(redshift_cosmos > limit_z_sdss)]
upper_cosmos_ra         = upper_cosmos['cosm_ra']
upper_cosmos_dec        = upper_cosmos['cosm_dec']
upper_cosmos_z          = upper_cosmos['cosm_z']
upper_cosmos_z_e        = np.zeros_like(upper_cosmos_L)
upper_cosmos_f_20cm     = cosmos_14GHz_flux[np.array(redshift_cosmos > limit_z_sdss)]  # mJy
upper_cosmos_f_20cm_e   = cosmos_14GHz_flux_e[np.array(redshift_cosmos > limit_z_sdss)]  # mJy
upper_cosmos_f_250GHz   = np.zeros_like(upper_cosmos_L)
upper_cosmos_f_250GHz_e = np.zeros_like(upper_cosmos_L)
upper_cosmos_L_250GHz   = np.zeros_like(upper_cosmos_L)
upper_cosmos_L_250GHz_e = np.zeros_like(upper_cosmos_L)
upper_cosmos_mass_1450  = np.zeros_like(upper_cosmos_L)

Arrays from `radio`catalog (`Heasarc`)

In [21]:
radio_sdss_ra         = radio_sources['sdss_ra']
radio_sdss_dec        = radio_sources['sdss_dec']
radio_sdss_u_lim      = np.zeros_like(L_21cm_radio, dtype=np.bool)
radio_sdss_z          = radio_sources['sdss_z']
radio_sdss_z_e        = radio_sources['sdss_z_err']
radio_sdss_f_20cm     = radio_sources['radio_flux']
radio_sdss_f_20cm_e   = radio_sources['radio_flux_err']
radio_sdss_f_250GHz   = np.zeros_like(L_21cm_radio)
radio_sdss_f_250GHz_e = np.zeros_like(L_21cm_radio)
radio_sdss_L_250GHz   = np.zeros_like(L_21cm_radio)
radio_sdss_L_250GHz_e = np.zeros_like(L_21cm_radio)
radio_sdss_mass_1450  = np.zeros_like(L_21cm_radio)

Merging `SDSS+FIRST` and `radio`

In [22]:
radio_large_sample_ra         = np.append(upper_sdss_ra,         radio_sdss_ra)          # deg
radio_large_sample_dec        = np.append(upper_sdss_dec,        radio_sdss_dec)         # deg
radio_large_sample_L          = np.append(upper_sdss_L,          L_21cm_radio)           # W/Hz
radio_large_sample_L_e        = np.append(upper_sdss_L_e,        L_21cm_radio_e)         # W/Hz
radio_large_sample_u_lim      = np.append(upper_sdss_u_lim,      radio_sdss_u_lim)
radio_large_sample_L_250GHz   = np.append(upper_sdss_L_250GHz,   radio_sdss_L_250GHz)    # W/Hz
radio_large_sample_L_250GHz_e = np.append(upper_sdss_L_250GHz_e, radio_sdss_L_250GHz_e)  # W/Hz
radio_large_sample_f_20cm     = np.append(upper_sdss_f_20cm,     radio_sdss_f_20cm)      # mJy
radio_large_sample_f_20cm_e   = np.append(upper_sdss_f_20cm_e,   radio_sdss_f_20cm_e)    # mJy
radio_large_sample_f_250GHz   = np.append(upper_sdss_f_250GHz,   radio_sdss_f_250GHz)    # mJy
radio_large_sample_f_250GHz_e = np.append(upper_sdss_f_250GHz_e, radio_sdss_f_250GHz_e)  # mJy
radio_large_sample_z          = np.append(upper_sdss_z,          radio_sdss_z)
radio_large_sample_z_e        = np.append(upper_sdss_z_e,        radio_sdss_z_e)
radio_large_sample_mass_1450  = np.append(upper_sdss_mass_1450,  radio_sdss_mass_1450)   # M_sun

Merging `SDSS+FIRST`+`radio` and the catalog from **Inayoshi et al., 2020**

In [23]:
medium_sample_ra        = np.append(radio_large_sample_ra,         high_z_ra[np.array(high_z_lum_14GHz>0)])               # deg
medium_sample_dec       = np.append(radio_large_sample_dec,        high_z_dec[np.array(high_z_lum_14GHz>0)])              # deg
medium_sample_L         = np.append(radio_large_sample_L,          high_z_lum_14GHz[np.array(high_z_lum_14GHz>0)])        # W/Hz
medium_sample_L_e       = np.append(radio_large_sample_L_e,        high_z_lum_14GHz_e[np.array(high_z_lum_14GHz>0)])      # W/Hz
medium_sample_u_lim     = np.append(radio_large_sample_u_lim,      high_z_up_lim[np.array(high_z_lum_14GHz>0)])
medium_sample_L_250     = np.append(radio_large_sample_L_250GHz,   high_z_lum_250[np.array(high_z_lum_14GHz>0)])          # W/Hz
medium_sample_L_250_e   = np.append(radio_large_sample_L_250GHz_e, high_z_lum_250GHz_e[np.array(high_z_lum_14GHz>0)])     # W/Hz
medium_sample_f20cm     = np.append(radio_large_sample_f_20cm,     high_z_14[np.array(high_z_lum_14GHz>0)] * 1e-3)        # mJy
medium_sample_f20cm_e   = np.append(radio_large_sample_f_20cm_e,   high_z_14_e[np.array(high_z_lum_14GHz>0)] * 1e-3)      # mJy
medium_sample_f250GHz   = np.append(radio_large_sample_f_250GHz,   high_z_250GHz[np.array(high_z_lum_14GHz>0)] * 1e-3)    # mJy
medium_sample_f250GHz_e = np.append(radio_large_sample_f_250GHz_e, high_z_250GHz_e[np.array(high_z_lum_14GHz>0)] * 1e-3)  # mJy
medium_sample_z         = np.append(radio_large_sample_z,          high_z_zs[np.array(high_z_lum_14GHz>0)])
medium_sample_z_e       = np.append(radio_large_sample_z_e,        high_z_zs_e[np.array(high_z_lum_14GHz>0)])
medium_sample_mass_1450 = np.append(radio_large_sample_mass_1450,  high_z_mass_1450[np.array(high_z_lum_14GHz>0)])        # M_sun

Merging `SDSS+FIRST`+`radio`+**Inayoshi et al., 2020** and `COSMOS` data

In [24]:
large_sample_ra        = np.append(medium_sample_ra,        upper_cosmos_ra)          # deg
large_sample_dec       = np.append(medium_sample_dec,       upper_cosmos_dec)         # deg
large_sample_L         = np.append(medium_sample_L,         upper_cosmos_L)           # W/Hz
large_sample_L_e       = np.append(medium_sample_L_e,       upper_cosmos_L_e)         # W/Hz
large_sample_u_lim     = np.append(medium_sample_u_lim,     upper_cosmos_u_lim)
large_sample_L_250     = np.append(medium_sample_L_250,     upper_cosmos_L_250GHz)    # W/Hz
large_sample_L_250_e   = np.append(medium_sample_L_250_e,   upper_cosmos_L_250GHz_e)  # W/Hz
large_sample_f20cm     = np.append(medium_sample_f20cm,     upper_cosmos_f_20cm)      # mJy
large_sample_f20cm_e   = np.append(medium_sample_f20cm_e,   upper_cosmos_f_20cm_e)    # mJy
large_sample_f250GHz   = np.append(medium_sample_f250GHz,   upper_cosmos_f_250GHz)    # mJy
large_sample_f250GHz_e = np.append(medium_sample_f250GHz_e, upper_cosmos_f_250GHz_e)  # mJy
large_sample_z         = np.append(medium_sample_z,         upper_cosmos_z)
large_sample_z_e       = np.append(medium_sample_z_e,       upper_cosmos_z_e)
large_sample_mass_1450 = np.append(medium_sample_mass_1450, upper_cosmos_mass_1450)   # M_sun

At this point, we also want to obtain more properties from the selected  
sources (**Inayoshi et al., 2020** + **SDSS+FIRST**). We will use `astroquery` to  
obtain information from `simbad`.

First, we obtain the names and coordinates of our sources to query them.

In [25]:
names_radio_sdss    = np.append(radio_sources['sdss_name'], upper_sdss['name'])
medium_sample_names = np.append(names_radio_sdss, upper_cosmos['cosm_name'])
large_sample_names  = np.append(medium_sample_names, high_z_names[np.array(high_z_lum_14GHz>0)])
large_sample_coords = SkyCoord(ra=large_sample_ra, dec=large_sample_dec, unit=(u.deg, u.deg))

Now, we correct/change the names for those which can be looked up in `Simbad` and `Ned`.

In [26]:
old_to_replace = ['SDSS J125507.61+463126.5', 'SDSS J160558.86+474300.1', 'SDSS J111036.32+481752.3', 'SDSS J163033.89+401209.6']
new_to_replace = ['NVSS J125507+463128', '2MASS J16055893+4742596', 'NVSS J111036+481753', 'SDSS J163033.90+401209.6']

for old, new in zip(old_to_replace, new_to_replace):
    index = np.where(large_sample_names == old)
    large_sample_names[index] = new

Then, we can query the database to obtain the desired data.  In this point,  
we also add more columns to be queried.

In [27]:
query_simbad_flag               = False
load_simbad_flag                = False
create_simbad_inayoshi_flag     = False
read_simbad_inayoshi_flag       = True
query_ned_names_flag            = True
query_ned_photometry_flag       = True
order_ned_photometry_flag       = True
create_simbad_inayoshi_ned_flag = True

In [28]:
if query_simbad_flag:
    customSimbad   = Simbad()
    initial_fields = customSimbad.get_votable_fields()

    if 'coordinates' in initial_fields:
        customSimbad.remove_votable_fields('coordinates')
        customSimbad.add_votable_fields('ra(d)', 'dec(d)')
    if 'z_value' not in initial_fields:
        customSimbad.add_votable_fields('z_value')
    for band in ['B','V','R','I','J','K']:
        if f'fluxdata({band})' not in initial_fields:
            customSimbad.add_votable_fields(f'flux({band})', f'flux_error({band})')

All sources but those from `COSMOS` catalog have meaningful (for `simbad`) names.  
Thus, separate queries will be executed. And, to standardize results, queries  
will only be based on coordinates (not names).

In [29]:
if query_simbad_flag:
    limit_set            = int(np.floor(np.shape(medium_sample_L)[0]/3))

In [30]:
if query_simbad_flag:
    result_table_simbad  = Table()

In [31]:
# result_table = customSimbad.query_objects(large_sample_names)
# customSimbad.TIMEOUT = 240

# result_table_job_a   = customSimbad.query_objects(large_sample_names[:limit_set])

# result_table_job_b   = customSimbad.query_objects(large_sample_names[limit_set:(limit_set*2)])

# result_table_job_c   = customSimbad.query_objects(large_sample_names[(limit_set*2):(limit_set*3)])

In [32]:
if query_simbad_flag:
    query_error = 0
    final_error = 0

In [33]:
if query_simbad_flag:
    with progressbar.ProgressBar(min_value=0, max_value=(limit_set - 1)) as bar:
        for index, coord in enumerate(large_sample_coords[:limit_set]):
            temp_table      = Table()
            try:
                temp_table      = customSimbad.query_region(coord, radius=2.0*u.arcsec)
                if len(temp_table) == 0:
                    temp_table      = Table(names=('RA_d', 'DEC_d'), dtype=(result_table_simbad['RA_d'].info.dtype, result_table_simbad['DEC_d'].info.dtype))
                    temp_table.add_row((coord.ra.deg, coord.dec.deg))
                temp_table.meta['description'] = 'Simbad_q'
            except:
                temp_table      = Table(names=('RA_d', 'DEC_d'), dtype=(result_table_simbad['RA_d'].info.dtype, result_table_simbad['DEC_d'].info.dtype))
                temp_table.add_row((coord.ra.deg, coord.dec.deg))
                temp_table.meta['description'] = 'Simbad_q'
                #print(f'Error with element {(index)} of the sample')
                query_error += 1
            try:
                if temp_table['RA_d'].unit != 'deg':
                    replacement_col     = MaskedColumn(temp_table['RA_d'].data, dtype=result_table_simbad['RA_d'].dtype, unit=result_table_simbad['RA_d'].unit, format=result_table_simbad['RA_d'].format, description=result_table_simbad['RA_d'].description)
                    temp_table['RA_d']  = replacement_col
                result_table_simbad = vstack([result_table_simbad, temp_table[0]])
            except:
                print(f'Error with element {(index)} of the sample')
                final_error += 1
            bar.update(index)

In [34]:
if query_simbad_flag:
    with progressbar.ProgressBar(min_value=limit_set, max_value=(limit_set*2 - 1)) as bar:
        for index, coord in enumerate(large_sample_coords[limit_set:(limit_set*2)]):
            temp_table      = Table()
            try:
                temp_table      = customSimbad.query_region(coord, radius=2.0*u.arcsec)
                if len(temp_table) == 0:
                    temp_table      = Table(names=('RA_d', 'DEC_d'), dtype=(result_table_simbad['RA_d'].info.dtype, result_table_simbad['DEC_d'].info.dtype))
                    temp_table.add_row((coord.ra.deg, coord.dec.deg))
                temp_table.meta['description'] = 'Simbad_q'
            except:
                temp_table      = Table(names=('RA_d', 'DEC_d'), dtype=(result_table_simbad['RA_d'].info.dtype, result_table_simbad['DEC_d'].info.dtype))
                temp_table.add_row((coord.ra.deg, coord.dec.deg))
                temp_table.meta['description'] = 'Simbad_q'
                #print(f'Error with element {(index + limit_set)} of the sample')
                query_error += 1
            try:
                if temp_table['RA_d'].unit != 'deg':
                    replacement_col     = MaskedColumn(temp_table['RA_d'].data, dtype=result_table_simbad['RA_d'].dtype, unit=result_table_simbad['RA_d'].unit, format=result_table_simbad['RA_d'].format, description=result_table_simbad['RA_d'].description)
                    temp_table['RA_d']  = replacement_col
                result_table_simbad = vstack([result_table_simbad, temp_table[0]])
            except:
                print(f'Error with element {(index + limit_set)} of the sample')
                final_error += 1
            bar.update(index + limit_set)

In [35]:
if query_simbad_flag:
    with progressbar.ProgressBar(min_value=(limit_set*2), max_value=(limit_set*3)) as bar:
        for index, coord in enumerate(large_sample_coords[(limit_set*2):(limit_set*3 + 1)]):
            temp_table      = Table()
            try:
                temp_table      = customSimbad.query_region(coord, radius=2.0*u.arcsec)
                if len(temp_table) == 0:
                    temp_table      = Table(names=('RA_d', 'DEC_d'), dtype=(result_table_simbad['RA_d'].info.dtype, result_table_simbad['DEC_d'].info.dtype))
                    temp_table.add_row((coord.ra.deg, coord.dec.deg))
                temp_table.meta['description'] = 'Simbad_q'
            except:
                temp_table      = Table(names=('RA_d', 'DEC_d'), dtype=(result_table_simbad['RA_d'].info.dtype, result_table_simbad['DEC_d'].info.dtype))
                temp_table.add_row((coord.ra.deg, coord.dec.deg))
                temp_table.meta['description'] = 'Simbad_q'
                # print(f'Error with element {(index + limit_set*2)} of the sample')
                query_error += 1
            try:
                if temp_table['RA_d'].unit != 'deg':
                    replacement_col     = MaskedColumn(temp_table['RA_d'].data, dtype=result_table_simbad['RA_d'].dtype, unit=result_table_simbad['RA_d'].unit, format=result_table_simbad['RA_d'].format, description=result_table_simbad['RA_d'].description)
                    temp_table['RA_d']  = replacement_col
                result_table_simbad = vstack([result_table_simbad, temp_table[0]])
            except:
                print(f'Error with element {(index + limit_set*2)} of the sample')
                final_error += 1
            bar.update(index + limit_set*2)

Query for `COSMOS`

In [36]:
if query_simbad_flag:
    limit_set_cosmos = int(np.floor(np.shape(upper_cosmos_L)[0]/3))

In [37]:
if query_simbad_flag:
    with progressbar.ProgressBar(min_value=(limit_set*3 + 1), max_value=(limit_set*3 + limit_set_cosmos)) as bar:
        for index, coord in enumerate(large_sample_coords[(limit_set*3 + 1):(limit_set*3 + 1 + limit_set_cosmos)]):
            temp_table      = Table()
            try:
                temp_table      = customSimbad.query_region(coord, radius=3.0*u.arcsec)
                if len(temp_table) == 0:
                    temp_table      = Table(names=('RA_d', 'DEC_d'), dtype=(result_table_simbad['RA_d'].info.dtype, result_table_simbad['DEC_d'].info.dtype))
                    temp_table.add_row((coord.ra.deg, coord.dec.deg))
                temp_table.meta['description'] = 'Simbad_q'
            except:
                # print(f'Error with element {(index + limit_set*3 + 1)} of the sample')
                temp_table      = Table(names=('RA_d', 'DEC_d'), dtype=(result_table_simbad['RA_d'].info.dtype, result_table_simbad['DEC_d'].info.dtype))
                temp_table.add_row((coord.ra.deg, coord.dec.deg))
                temp_table.meta['description'] = 'Simbad_q'
                query_error += 1
            try:
                if temp_table['RA_d'].unit != 'deg':
                    replacement_col     = MaskedColumn(temp_table['RA_d'].data, dtype=result_table_simbad['RA_d'].dtype, unit=result_table_simbad['RA_d'].unit, format=result_table_simbad['RA_d'].format, description=result_table_simbad['RA_d'].description)
                    temp_table['RA_d']  = replacement_col
                result_table_simbad = vstack([result_table_simbad, temp_table[0]])
            except:
                print(f'Error with element {(index + limit_set*3 + 1)} of the sample')
                temp_table      = Table(names=('RA_d', 'DEC_d'), dtype=(result_table_simbad['RA_d'].info.dtype, result_table_simbad['DEC_d'].info.dtype))
                temp_table.add_row((coord.ra.deg, coord.dec.deg))
                temp_table.meta['description'] = 'Simbad query for radio objects'
                result_table_simbad = vstack([result_table_simbad, temp_table[0]])
                final_error += 1
            bar.update(index + limit_set*3 + 1)

In [38]:
if query_simbad_flag:
    with progressbar.ProgressBar(min_value=(limit_set*3 + 1 + limit_set_cosmos), max_value=(limit_set*3 + limit_set_cosmos*2)) as bar:
        for index, coord in enumerate(large_sample_coords[(limit_set*3 + 1 + limit_set_cosmos):(limit_set*3 + 1 + limit_set_cosmos*2)]):
            temp_table      = Table()
            try:
                temp_table      = customSimbad.query_region(coord, radius=3.0*u.arcsec)
                if len(temp_table) == 0:
                    temp_table      = Table(names=('RA_d', 'DEC_d'), dtype=(result_table_simbad['RA_d'].info.dtype, result_table_simbad['DEC_d'].info.dtype))
                    temp_table.add_row((coord.ra.deg, coord.dec.deg))
                temp_table.meta['description'] = 'Simbad_q'
            except:
                temp_table      = Table(names=('RA_d', 'DEC_d'), dtype=(result_table_simbad['RA_d'].info.dtype, result_table_simbad['DEC_d'].info.dtype))
                temp_table.add_row((coord.ra.deg, coord.dec.deg))
                temp_table.meta['description'] = 'Simbad_q'
                # print(f'Error with element {(index + limit_set*3 + 1 + limit_set_cosmos)} of the sample')
                query_error += 1
            try:
                if temp_table['RA_d'].unit != 'deg':
                    replacement_col     = MaskedColumn(temp_table['RA_d'].data, dtype=result_table_simbad['RA_d'].dtype, unit=result_table_simbad['RA_d'].unit, format=result_table_simbad['RA_d'].format, description=result_table_simbad['RA_d'].description)
                    temp_table['RA_d']  = replacement_col
                result_table_simbad = vstack([result_table_simbad, temp_table[0]])
            except:
                print(f'Error with element {(index + limit_set*3 + 1 + limit_set_cosmos)} of the sample')
                temp_table      = Table(names=('RA_d', 'DEC_d'), dtype=(result_table_simbad['RA_d'].info.dtype, result_table_simbad['DEC_d'].info.dtype))
                temp_table.add_row((coord.ra.deg, coord.dec.deg))
                temp_table.meta['description'] = 'Simbad query for radio objects'
                result_table_simbad = vstack([result_table_simbad, temp_table[0]])
                final_error += 1
            bar.update(index + limit_set*3 + 1 + limit_set_cosmos)

In [39]:
if query_simbad_flag:
    with progressbar.ProgressBar(min_value=(limit_set*3 + 1 + limit_set_cosmos*2), max_value=np.shape(large_sample_coords)[0]) as bar:
        for index, coord in enumerate(large_sample_coords[(limit_set*3 + 1 + limit_set_cosmos*2):]):
            temp_table      = Table()
            try:
                temp_table      = customSimbad.query_region(coord, radius=3.0*u.arcsec)
                if len(temp_table) == 0:
                    temp_table      = Table(names=('RA_d', 'DEC_d'), dtype=(result_table_simbad['RA_d'].info.dtype, result_table_simbad['DEC_d'].info.dtype))
                    temp_table.add_row((coord.ra.deg, coord.dec.deg))
                temp_table.meta['description'] = 'Simbad_q'
            except:
                temp_table      = Table(names=('RA_d', 'DEC_d'), dtype=(result_table_simbad['RA_d'].info.dtype, result_table_simbad['DEC_d'].info.dtype))
                temp_table.add_row((coord.ra.deg, coord.dec.deg))
                temp_table.meta['description'] = 'Simbad_q'
                # print(f'Error with element {(index + limit_set*3 + 1 + limit_set_cosmos*2)} of the sample')
                query_error += 1
            try:
                if temp_table['RA_d'].unit != 'deg':
                    replacement_col     = MaskedColumn(temp_table['RA_d'].data, dtype=result_table_simbad['RA_d'].dtype, unit=result_table_simbad['RA_d'].unit, format=result_table_simbad['RA_d'].format, description=result_table_simbad['RA_d'].description)
                    temp_table['RA_d']  = replacement_col
                result_table_simbad = vstack([result_table_simbad, temp_table[0]])
            except:
                print(f'Error with element {(index + limit_set*3 + 1 + limit_set_cosmos*2)} of the sample')
                temp_table      = Table(names=('RA_d', 'DEC_d'), dtype=(result_table_simbad['RA_d'].info.dtype, result_table_simbad['DEC_d'].info.dtype))
                temp_table.add_row((coord.ra.deg, coord.dec.deg))
                temp_table.meta['description'] = 'Simbad query for radio objects'
                result_table_simbad = vstack([result_table_simbad, temp_table[0]])
                final_error += 1
            bar.update(index + limit_set*3 + 1 + limit_set_cosmos*2)

We save the query to a file for future executions.

In [40]:
if query_simbad_flag:
    result_table_simbad.info

In [41]:
if query_simbad_flag:
    result_table_simbad.write('large_cat_simbad_query.csv', format='ascii.csv', overwrite=True, serialize_method='data_mask')

Steps to create a copy of table to save it as `fits` file.

In [42]:
if query_simbad_flag:
    copy_simbad_to_write = result_table_simbad
    str_id = copy_simbad_to_write['MAIN_ID'].astype('str')
    copy_simbad_to_write.replace_column('MAIN_ID', str_id)

In [43]:
if query_simbad_flag:
    copy_simbad_to_write.info

In [44]:
if query_simbad_flag:
    copy_simbad_to_write.write('large_cat_simbad_query.fits', format='fits', overwrite=True, serialize_method='data_mask')

To save running time, we can load the data from a file.

In [45]:
#if load_simbad_flag:
#    result_table_simbad = Table.read(cat_path + 'large_cat_simbad_query.csv', format='ascii.csv')

In [46]:
if load_simbad_flag:
    result_table_simbad     = Table.read(cat_path + 'large_cat_simbad_query.fits');

In [47]:
# result_table_simbad.info

From this point, we merge the data from the query to `simbad` with the  
values from this notebook (**Inayoshi et al., 2020** and **SDSS+FIRST**).  
In order to do this, we convert the data into `astropy` columns, and then  
into `astropy` tables. They will be ready to be exported.

In [48]:
if create_simbad_inayoshi_flag:
    column_cat_name     = MaskedColumn(large_sample_names, name='CAT_NAME', dtype='str', description='Name used in this catalog', mask=np.array(large_sample_names == ''))
    column_z_own        = MaskedColumn(large_sample_z, name='Z_OWN', unit='', description='Redshift from Inayoshi+2020 or SDSS+FIRST', fill_value=np.nan, mask=np.array(large_sample_z == 0))
    column_z_own_err    = MaskedColumn(large_sample_z_e, name='Z_OWN_ERR', unit='', description='Redshift error from Inayoshi+2020 or SDSS+FIRST', fill_value=np.nan, mask=np.array(large_sample_z_e == 0))
    column_L_14GHz      = MaskedColumn(large_sample_L, name='L_20CM', unit='W/Hz', description='Luminosity in 1.4 GHz', fill_value=np.nan, mask=np.array(large_sample_L == 0))
    column_L_14GHz_err  = MaskedColumn(large_sample_L_e, name='L_20CM_ERR', unit='W/Hz', description='Luminosity error in 1.4 GHz', fill_value=np.nan, mask=np.array(large_sample_L_e == 0))
    column_L_14GHz_up   = MaskedColumn(large_sample_u_lim, name='L_20CM_UP_LIM', dtype='bool', description='True if L_20CM is upper limit')
    column_L_250GHz     = MaskedColumn(large_sample_L_250, name='L_250GHZ', unit='W/Hz', description='Luminosity in 250 GHz', fill_value=np.nan, mask=np.array(large_sample_L_250 == 0))
    column_L_250GHz_err = MaskedColumn(large_sample_L_250_e, name='L_250GHZ_ERR', unit='W/Hz', description='Luminosity error in 250 GHz', fill_value=np.nan, mask=np.array(large_sample_L_250_e == 0))
    column_f_20cm       = MaskedColumn(large_sample_f20cm, name='F_20CM', unit='mJy', description='Flux in 20 cm', fill_value=np.nan, mask=np.array(large_sample_f20cm == 0))
    column_f_20cm_err   = MaskedColumn(large_sample_f20cm_e, name='F_20CM_ERR', unit='mJy', description='Flux error in 20 cm', fill_value=np.nan, mask=np.array(large_sample_f20cm_e == 0))
    column_f_250GHz     = MaskedColumn(large_sample_f250GHz, name='F_250GHZ', unit='mJy', description='Flux in 250 GHz', fill_value=np.nan, mask=np.array(large_sample_f250GHz == 0))
    column_f_250GHz_err = MaskedColumn(large_sample_f250GHz_e, name='F_250GHZ_ERR', unit='mJy', description='Flux error in 250 GHz', fill_value=np.nan, mask=np.array(large_sample_f250GHz_e == 0))
    column_mass_1450    = MaskedColumn(large_sample_mass_1450, name='MASS_1450', unit='Msun', description='Mass from mag_1450 (UV)', fill_value=np.nan, mask=np.array(large_sample_mass_1450 == 0))

In [49]:
if create_simbad_inayoshi_flag:
    result_table_simbad.add_columns([column_cat_name, column_z_own, column_z_own_err, column_L_14GHz, column_L_14GHz_err, column_L_14GHz_up, column_L_250GHz, column_L_250GHz_err, column_f_20cm, column_f_20cm_err, column_f_250GHz, column_f_250GHz_err, column_mass_1450])

In [50]:
#str_id = result_table['MAIN_ID'].astype('str')
#result_table.replace_column('MAIN_ID', str_id)

In [51]:
# copy_table = result_table_simbad.filled(fill_value=np.nan)

We write the table into a file. It can be `.fits`, `.votable`, etc.

In [52]:
#copy_table.write('high_z_qsos.ecsv', format='ascii.ecsv', overwrite=True, serialize_method='data_mask')

In [53]:
if create_simbad_inayoshi_flag:
    result_table_simbad.write('large_cat_simbad_query_inayoshi.fits', format='fits', overwrite=True, serialize_method='data_mask')

In [54]:
if create_simbad_inayoshi_flag:
    result_table_simbad.write('large_cat_simbad_query_inayoshi.csv', format='ascii.csv', overwrite=True, serialize_method='data_mask')

To save running time, we can load the data from a file.

In [55]:
#if read_simbad_inayoshi_flag:
#    result_table_simbad = Table.read(cat_path + 'large_cat_simbad_query_inayoshi.csv', format='ascii.csv')

In [55]:
if read_simbad_inayoshi_flag:
    result_table_simbad = Table.read(cat_path + 'large_cat_simbad_query_inayoshi.fits');

In [56]:
result_table_simbad.info

<Table length=18023>
     name      dtype    unit   format                    description                   n_bad
------------- ------- ------- -------- ----------------------------------------------- -----
      MAIN_ID bytes41                                    Main identifier for an object   606
         RA_d float64     deg {:11.8f}                                 Right ascension     0
        DEC_d float64     deg {:12.8f}                                     Declination     0
      Z_VALUE float64         {:16.7f}                                        Redshift  1644
       FLUX_B float32     mag                                              Magnitude B  8961
 FLUX_ERROR_B float32         {:12.3f}                                      flux error 11228
       FLUX_V float32     mag                                              Magnitude V  8857
 FLUX_ERROR_V float32         {:12.3f}                                      flux error 11189
       FLUX_R float32     mag                    

Query the objects of the table in other catalogs and services.

In [57]:
#from astroquery.heasarc import Heasarc
#Heasarc.query_mission_cols(mission='radio')
#tabb = Heasarc.query(large_sample_names, mission='radio', timeout=90)

In [58]:
customNed        = Ned()
fields_to_remove = ['No.', 'Photometry Measurement', 'Uncertainty', 'Units', 'Frequency', 'Significance', 'Published frequency', 'Frequency Mode', 'Coordinates Targeted', 'Spatial Mode', 'Qualifiers', 'Comments']

Querying sources with name in `Ned`.

In [59]:
# empty_counter = 0
# res_tab       = {}
# for name in large_sample_names:
#     try:
#         res_tab[name] = customNed.get_table(name, output_table_format=1)
#         res_tab[name].remove_columns(fields_to_remove)
#     except:
#         res_tab[name] = Table()
#         empty_counter += 1

As before, we can do it with coordinates.  

First, we query the coordinates. If we found something,  
we use the name of the source to obtain it photometry.

In [60]:
coords_simbad_inayoshi = SkyCoord(result_table_simbad['RA_d'], result_table_simbad['DEC_d'], unit=u.deg)

In [61]:
empty_tab_name_counter  = 0
empty_tab_photo_counter = 0
error_tab_name_counter  = 0
error_tab_photo_counter = 0
ned_tables              = {}
ned_info                = {}
ned_names               = []
#ned_names               = np.array([''  for x in np.arange(np.shape(large_sample_names)[0])])

In [62]:
if query_ned_names_flag:
    with progressbar.ProgressBar(min_value=0, max_value=np.shape(coords_simbad_inayoshi)[0]) as bar:
        for index, coord in enumerate(coords_simbad_inayoshi):
            try:
                init_table            = customNed.query_region(coords_simbad_inayoshi[index], radius=3.0*u.arcsec)
                if len(init_table) == 0:
                    init_table        = Table(names=('Object Name', 'RA', 'DEC'), dtype=('str', 'float', 'float'), masked=True)
                    init_table.add_row(('No Name', coords_simbad_inayoshi[index].ra.deg, coords_simbad_inayoshi[index].dec.deg), mask=[True, False, False])
                    ned_info[index]   = init_table
                    ned_names.append('No Name')
                    empty_tab_name_counter += 1
                    continue
                init_table.remove_columns(['Magnitude and Filter', 'Positions', 'Diameter Points'])
                ned_info[index]   = init_table
                used_source_idx   = np.nanargmin(init_table['Separation'])  # Index of element with lowest separation from coords
                init_name         = init_table['Object Name'][used_source_idx]
                # ned_names[index]  = init_name
                ned_names.append(init_name)
            except:
                init_table        = Table(names=('Object Name', 'RA', 'DEC'), dtype=('str', 'float', 'float'), masked=True)
                init_table.add_row(('No Name', coords_simbad_inayoshi[index].ra.deg, coords_simbad_inayoshi[index].dec.deg), mask=[True, False, False])
                ned_info[index]   = init_table
                ned_names.append('No Name')
                error_tab_name_counter += 1
            bar.update(index)

100% (18023 of 18023) |##################| Elapsed Time: 0:09:40 Time:  0:09:40


In [63]:
ned_names = np.array(ned_names)

In [64]:
ned_redshifts = []
for key in ned_info:
    ned_redshifts.append(ned_info[index]['Redshift'])
ned_redshifts = np.array(ned_redshifts)

In [65]:
counter_count = 0
indices_non   = []
for index, name in enumerate(ned_names):
    if name != 'No Name':
        counter_count += 1
for index, name in enumerate(ned_names):
    if name == 'No Name':
        indices_non.append(index)

In [66]:
if query_ned_photometry_flag:
    with progressbar.ProgressBar(min_value=0, max_value=np.shape(ned_names)[0]) as bar:
        for index, name in enumerate(ned_names):
            try:
                if name == 'No Name':
                    phot_table        = Table(names=('Observed Passband',), dtype=('str',), masked=True)
                    phot_table.add_row(('No Passband',), mask=(True,))
                    ned_tables[index] = phot_table
                    empty_tab_photo_counter += 1
                    continue
                phot_table            = customNed.get_table(name, table='photometry', output_table_format=3)
                phot_table.remove_columns(fields_to_remove)
                ned_tables[index]     = phot_table
            except:
                phot_table            = Table(names=('Observed Passband',), dtype=('str',), masked=True)
                phot_table.add_row(('No Passband',), mask=(True,))
                ned_tables[index]     = phot_table
                error_tab_photo_counter += 1
            bar.update(index)

100% (18023 of 18023) |##################| Elapsed Time: 0:07:45 Time:  0:07:45


In [67]:
ned_tables[12223].colnames

['Observed Passband',
 'Flux Density',
 'Upper limit of uncertainty',
 'Lower limit of uncertainty',
 'Upper limit of Flux Density',
 'Lower limit of Flux Density',
 'NED Uncertainty',
 'NED Units',
 'Refcode']

In [68]:
# empty_counter = 0
# res_tab       = {}
# for name in large_sample_names:
#     try:
#         res_tab[name] = customNed.query_region(large_sample_coords, radius=3.0*u.arcsec)[0]
#         res_tab[name].remove_columns(fields_to_remove)
#     except:
#         res_tab[name] = Table()
#         empty_counter += 1

In [71]:
if order_ned_photometry_flag:
    limit_set_ned = int(np.floor(np.shape(ned_names)[0]/4))

In [69]:
if order_ned_photometry_flag:
    temp_table_ned_photo             = Table()
    chunk_size                       = 300  # Number of elements to calculate before dumping results to external table
    with progressbar.ProgressBar(min_value=0, max_value=np.shape(ned_names)[0]) as bar:
        for index, source_name in enumerate(ned_names):  # Some names will be. 'No Name'
            band_names_str           = []
            column_names_str         = []
            measure_names            = ned_tables[index].colnames[1:]
            # init_table = Table(names=('MAIN_ID',), dtype=('str',), masked=True)
            cord_str                 = coords_simbad_inayoshi[index].to_string('decimal')
            init_table               = Table(data=np.array([cord_str, source_name]), names=('COORD', 'MAIN_ID'), dtype=('str', 'str'), masked=True)
            if source_name == 'No Name':
                # init_table = Table(('No Name',), names=('MAIN_ID',), dtype=('str',), masked=True)
                # init_table.add_row(('No Name',), mask=(True,)) # Mask values in the last step instead
                if index == 0 or (index % chunk_size == 1 and index > 1):
                    init_table_large     = Table(init_table)
                    bar.update(index)
                    continue
                init_table_large     = vstack([init_table_large, init_table])
                if index % chunk_size == 0 and index > 0:
                    temp_table_ned_photo = vstack([temp_table_ned_photo, init_table_large])
                    # result_table_simbad_copy = join(result_table_simbad_copy, init_table_large, keys='COORD', join_type='outer')
                bar.update(index)
                continue
            if len(measure_names) == 0:
                # init_table = Table(('No Name',), names=('MAIN_ID',), dtype=('str',), masked=True)
                # init_table.add_row((source_name,), mask=(True,))
                if index == 0 or (index % chunk_size == 1 and index > 1):
                    init_table_large     = Table(init_table)
                    bar.update(index)
                    continue
                init_table_large     = vstack([init_table_large, init_table])
                if index % chunk_size == 0 and index > 0:
                    temp_table_ned_photo = vstack([temp_table_ned_photo, init_table_large])
                    # result_table_simbad_copy = join(result_table_simbad_copy, init_table_large, keys='COORD', join_type='outer')
                bar.update(index)
                continue
            # init_table = Table((str(source_name),), names=('MAIN_ID',), dtype=('str',), masked=True)
            # init_table.add_row((str(source_name),))
            for row in ned_tables[index]:
                band_name_str        = str(row['Observed Passband'].decode('utf-8'))
                if str(band_name_str) not in band_names_str:
                    band_names_str.append(str(band_name_str))
                    column_name_flux = 'Flux Density ' + band_name_str
                    column_name_err  = 'NED Uncertainty ' + band_name_str
                    if column_name_flux not in column_names_str:
                        column_names_str.append(column_name_flux)
                        column_flux  = MaskedColumn(row['Flux Density'], name=column_name_flux, unit=ned_tables[index]['Flux Density'].unit, dtype='float')
                        column_err   = MaskedColumn(row['NED Uncertainty'], name=column_name_err, dtype='str')
                        init_table.add_columns((column_flux, column_err))
            #init_table.remove_column('MAIN_ID')
            if index == 0 or (index % chunk_size == 1 and index > 1):
                init_table_large     = Table(init_table)
                bar.update(index)
                continue
            init_table_large         = vstack([init_table_large, init_table])
            if index % chunk_size == 0 and index > 0:
                temp_table_ned_photo = vstack([temp_table_ned_photo, init_table_large])
                # result_table_simbad_copy = join(result_table_simbad_copy, init_table_large, keys='COORD', join_type='outer')
            bar.update(index)
    temp_table_ned_photo = vstack([temp_table_ned_photo, init_table_large])
    # result_table_simbad_copy = join(result_table_simbad_copy, init_table_large, keys='COORD', join_type='outer')
    #result_table_simbad_copy         = Table(result_table_simbad)
    #result_table_simbad_copy.add_column(coords_simbad_inayoshi.to_string('decimal'), name='COORD')

100% (18023 of 18023) |##################| Elapsed Time: 0:56:45 Time:  0:56:45


In [70]:
len(temp_table_ned_photo.colnames)

1852

In [71]:
len(np.unique(coords_simbad_inayoshi.to_string('hmsdms'), return_index=True)[1])

17979

In [89]:
len(coords_simbad_inayoshi.to_string('hmsdms'))

18023

In [78]:
aaa, bbb, ccc, ddd = np.unique(temp_table_ned_photo['COORD'], return_index=True, return_inverse=True, return_counts=True)

In [88]:
np.where(ddd == 1)

(array([    0,     1,     2, ..., 17976, 17977, 17978]),)

In [86]:
np.where(temp_table_ned_photo['COORD'] == temp_table_ned_photo[1934]['COORD'])

(array([1934]),)

In [85]:
temp_table_ned_photo[1934]['COORD']

'199.505 8.9605'

In [73]:
temp_table_ned_photo['COORD'][np.unique(temp_table_ned_photo['COORD'], return_index=True)[1]]

0.00532383 -2.03326
0.1126 3.121
0.128255 1.87252
0.13136 -8.69582
0.182201 0.333873
0.210845 -10.3655
0.214856 0.200727
0.216824 -4.29527
0.229011 1.02873
0.247222 7.15
0.284773 12.7329


In [158]:
indices = np.array(ned_names == 'No Name')

In [183]:
aaa = np.array(np.unique(coords_simbad_inayoshi.to_string('decimal', precision=10), return_index=True)[1])

In [178]:
bbb = np.array(np.unique(test_copy_simbad['MAIN_ID'], return_index=True)[1])

In [184]:
len(aaa)

17979

In [177]:
len(np.unique(np.arange(18023)))

18023

In [188]:
coords_list = np.array(coords_simbad_inayoshi.to_string('decimal', precision=15))

In [186]:
np.unique(coords_list[indices])

array(['0.890486368971400 12.271402401646000',
       '1.583852646067900 12.394228118483101',
       '10.457596145485100 5.117498616294700',
       '11.934366786710200 3.852404365627400',
       '12.283999493272500 -5.527044373625600',
       '149.430010000000010 1.830740000000000',
       '149.483587999999997 2.318789000000000',
       '149.656838999999991 1.607648000000000',
       '149.666831000000002 1.796849000000000',
       '149.679290000000009 2.250490000000000',
       '149.721973999999989 1.670438000000000',
       '149.751955000000009 1.892047000000000',
       '149.865170000000006 1.968580000000000',
       '149.873057999999986 2.377983000000000',
       '149.893570000000011 1.792930000000000',
       '149.904669000000013 2.863645000000000',
       '150.135257999999993 1.944202000000000',
       '150.148514000000006 2.881495000000000',
       '150.223264000000000 1.801835000000000',
       '150.288458999999989 1.682514000000000',
       '150.346552000000003 1.67132300000000

In [172]:
len(np.unique(coords_list))

17979

In [149]:
len(coords_simbad_inayoshi)

18023

In [143]:
len(result_table_simbad)

18023

In [117]:
test_copy_simbad = Table(result_table_simbad)

In [128]:
coord_col = MaskedColumn(coords_simbad_inayoshi.to_string('decimal'), name='COORD', dtype='str')

In [129]:
test_copy_simbad.add_column(coord_col)

In [133]:
test_copy_ned    = Table(temp_table_ned_photo)
test_copy_ned.remove_column('MAIN_ID')

In [134]:
test_copy_stack  = join(test_copy_simbad, test_copy_ned, keys='COORD', join_type='outer')

In [142]:
len(test_copy_stack)

18141

In [195]:
test_copy_stack[999].meta['TTYPE1000']

KeyError: 'TTYPE1000'

In [ ]:
test_copy_stack.info

In [197]:
if create_simbad_inayoshi_ned_flag:
    test_copy_stack.write('large_cat_simbad_inayoshi_ned.hdf5', format='hdf5', overwrite=True, serialize_method='data_mask', serialize_meta=True)

ValueError: Unable to create dataset (object header message is too large)

In [ ]:
if create_simbad_inayoshi_ned_flag:
    test_copy_stack.write('large_cat_simbad_inayoshi_ned.fits', format='fits', overwrite=True, serialize_method='data_mask')

In [135]:
len(np.unique(test_copy_stack.colnames))

1880

In [136]:
test_copy_stack.colnames

['MAIN_ID',
 'RA_d',
 'DEC_d',
 'Z_VALUE',
 'FLUX_B',
 'FLUX_ERROR_B',
 'FLUX_V',
 'FLUX_ERROR_V',
 'FLUX_R',
 'FLUX_ERROR_R',
 'FLUX_I',
 'FLUX_ERROR_I',
 'FLUX_J',
 'FLUX_ERROR_J',
 'FLUX_K',
 'FLUX_ERROR_K',
 'CAT_NAME',
 'Z_OWN',
 'Z_OWN_ERR',
 'L_20CM',
 'L_20CM_ERR',
 'L_20CM_UP_LIM',
 'L_250GHZ',
 'L_250GHZ_ERR',
 'F_20CM',
 'F_20CM_ERR',
 'F_250GHZ',
 'F_250GHZ_ERR',
 'MASS_1450',
 'COORD',
 'Flux Density u (SDSS PSF) AB',
 'NED Uncertainty u (SDSS PSF) AB',
 'Flux Density g (SDSS PSF) AB',
 'NED Uncertainty g (SDSS PSF) AB',
 'Flux Density r (SDSS PSF) AB',
 'NED Uncertainty r (SDSS PSF) AB',
 'Flux Density i (SDSS PSF) AB',
 'NED Uncertainty i (SDSS PSF) AB',
 'Flux Density z (SDSS PSF) AB',
 'NED Uncertainty z (SDSS PSF) AB',
 'Flux Density W1 (WISE)',
 'NED Uncertainty W1 (WISE)',
 'Flux Density W2 (WISE)',
 'NED Uncertainty W2 (WISE)',
 'Flux Density W3 (WISE)',
 'NED Uncertainty W3 (WISE)',
 'Flux Density W4 (WISE)',
 'NED Uncertainty W4 (WISE)',
 'Flux Density 1700 A (SD

In [84]:
temp_table_ned_photo[0]['MAIN_ID']

'SDSS J001117.36-104530.9'

In [105]:
init_table_large['MAIN_ID']

SSTSL2 J095858.88+025619.8
SSTSL2 J100157.28+025622.0
WISEA J100001.35+025620.7
SSTSL2 J100314.18+025623.5
WISEA J100135.66+025624.0
WISEA J100020.35+025623.5
SSTSL2 J095809.15+025627.9
WISEA J100214.46+025630.3
SSTSL2 J095847.47+025632.9
WISEA J100148.25+025632.8
SSTSL2 J095841.57+025632.9


In [ ]:
for row in ned_tables[index]:
    print(row['Flux Density'])

In [ ]:
# result_table_copy['MAIN_ID', '1.4_GHz_(VLA)_Flux_Density', '1.4GHz_Flux_Density'].show_in_browser(jsviewer=True)
# result_table_copy['MAIN_ID', 'CAT_NAME', '1.4_GHz_(VLA)_Flux_Density', '1.4GHz_Flux_Density'].show_in_browser(jsviewer=True)
# result_table_copy['1.4*'].show_in_browser(jsviewer=True)

In [ ]:
# result_table_copy.info

In [ ]:
result_table_copy['1.4_GHz_(VLA)_Flux_Density', '1.4GHz_Flux_Density'].info(['attributes', 'stats'])

In [ ]:
# for name in large_sample_names:
#     print(name)

IDs to be changed for use in NED

OLD                          NEW

SDSS J125507.61+463126.5     NVSS J125507+463128  
SDSS J160558.86+474300.1     2MASS J16055893+4742596  
SDSS J111036.32+481752.3     NVSS J111036+481753  
SDSS J163033.89+401209.6     SDSS J163033.90+401209.6

In [ ]:
52 - empty_counter

In [ ]:
result_table_copy.write('high_z_qsos_large.csv', format='ascii.csv', overwrite=True, serialize_method='data_mask')

In [ ]:
data_z_L14 = np.array([result_table_copy['Z_OWN'], result_table_copy['Z_OWN_ERR'], result_table_copy['L_20CM'], result_table_copy['L_20CM_ERR'], result_table_copy['L_20CM_UP_LIM']])

In [ ]:
data_z_L14 = np.array([column_z_own, column_z_own_err, column_L_14GHz, column_L_14GHz_err, column_L_14GHz_up])

In [ ]:
np.shape(data_z_L14)

In [ ]:
z_range        = np.arange(0, 10.2, 0.25)  # array to be used in other instances
lum_dist_range = luminosity_distance(z_range)* 3.086e22  # in m

In [ ]:
lum_FIRST   = 4 * np.pi * lum_dist_range**2 * 1e-3  * 1e-26 * (1 + z_range)**(alpha_used - 1)
lum_50uJy   = 4 * np.pi * lum_dist_range**2 * 50e-6 * 1e-26 * (1 + z_range)**(alpha_used - 1)
lum_20uJy   = 4 * np.pi * lum_dist_range**2 * 20e-6 * 1e-26 * (1 + z_range)**(alpha_used - 1)
lum_10uJy   = 4 * np.pi * lum_dist_range**2 * 10e-6 * 1e-26 * (1 + z_range)**(alpha_used - 1)
lum_1uJy    = 4 * np.pi * lum_dist_range**2 * 1e-6  * 1e-26 * (1 + z_range)**(alpha_used - 1)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patheffects as mpe
import matplotlib.patheffects as path_effects

In [ ]:
cmap        = cm.get_cmap('inferno')
dark_color  = cmap(0.4)  # RGBA color code
light_color = cmap(0.6)  # RGBA color code
colors      = [cmap(0.15), cmap(0.30), cmap(0.45), cmap(0.60), cmap(0.75), cmap(0.90)]
colors_8    = [cmap(0.12), cmap(0.24), cmap(0.35), cmap(0.48), cmap(0.6), cmap(0.72), cmap(0.84), cmap(0.96)]

In [ ]:
pe1 = [mpe.Stroke(linewidth=3, foreground='black'),
       mpe.Stroke(foreground='white', alpha=1),
       mpe.Normal()]

alp_str     = r'$\alpha$'

In [ ]:
fig = plt.figure(figsize=(10,7))
ax1 = fig.add_subplot(111)

ax1.errorbar(data_z_L14[0, :], data_z_L14[2,:], xerr=data_z_L14[1, :], yerr=data_z_L14[3, :], marker='P', ls='', color=colors[3], markersize=10, zorder=7, alpha=0.75, uplims=data_z_L14[4, :], elinewidth=2, mec='k', label='Radio QSO')

#ax1.errorbar(high_z_zs[filter_250GHz], lum_250_new_alpha_rescale[filter_250GHz], yerr=lum_250_new_alpha_rescale_e[filter_250GHz], marker='P', ls='', color=colors[4], markersize=10, zorder=7, alpha=0.65, uplims=high_z_up_lim[filter_250GHz], elinewidth=2, mec='k', label='mm QSO')

ax1.plot(z_range, lum_1uJy, label='1 $\mu$Jy', color=colors[0], path_effects=pe1)
# ax1.plot(z_range, lum_10uJy, label='10 $\mu$Jy', color=colors[1], path_effects=pe1)
ax1.plot(z_range, lum_20uJy, label='20 $\mu$Jy', color=colors[1], path_effects=pe1)
ax1.plot(z_range, lum_50uJy, label='50 $\mu$Jy', color=colors[3], path_effects=pe1)
ax1.plot(z_range, lum_FIRST, label='FIRST', color=colors[5], path_effects=pe1)
#ax1.plot(z_range, SFR_3000, label='3000M_sun/yr', color=colors[2], path_effects=pe1)
#ax1.fill_between(z_range, SFR_3000 - SFR_3000_e, SFR_3000 + SFR_3000_e, color=colors[2], fc=colors[2], alpha=0.3, zorder=6)
#gradient_fill(z_range, max_pow_sfr_fix + 3 * max_pow_sfr_fix_e, fill_color=colors[4], color=colors[4], label='SFR - fixed M', zorder=5, ax=ax1)
#ax1.plot(z_range, max_pow_sfr_fix, '-', color=colors[4], alpha=0.45)
#gradient_fill(z_range, max_pow_sfr + 3 * max_pow_sfr_e, fill_color=colors[3], color=colors[3], label='SFR - evolv. M', zorder=5, ax=ax1)
#ax1.plot(z_range, max_pow_sfr, '-', color=colors[3], alpha=0.45)
ax1.set_xlabel('$z$', size=14)
ax1.set_ylabel('$\mathrm{L}_{1.4 \mathrm{GHz}} [\mathrm{W/Hz}]$', size=14)
ax1.set_yscale('log')
ax1.set_ylim((1e23, 1e29))
ax1.set_xlim(left=-0.001, right=10)
#ax1.set_xlim(left=5.5, right=8)
ax1.legend(loc=1, title=f'{alp_str}={alpha_used}', ncol=2, labelspacing=0.5, borderpad=0.5, columnspacing=0.5, handletextpad=0.5)


ax2 = ax1.twinx()
ax2.set_ylabel('$\mathrm{SFR}\, [\mathrm{M}_{\odot} \mathrm{yr}^{-1}]$', size=14)
ax2.set_ylim((1e23 * 5.9e-22, 1e29 * 5.9e-22))
ax2.set_yscale('log')
ax2.figure.canvas.draw()

#plt.setp(ax2.get_yticklabels(), visible=False)
ax1.tick_params(which='both',top=True, right=False, direction='in')
ax2.tick_params(which='both',top=True, right=True, direction='in', labelright=True)

ax1.tick_params(axis='both', which='major', labelsize=14)
ax2.tick_params(axis='both', which='major', labelsize=14)
fig.tight_layout(w_pad=0.01)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from sklearn.cluster import OPTICS
np.random.seed(5)

In [ ]:
estimator = KMeans(n_clusters=6)  # Best suited for straight boundaries
estimator.fit(data_z_L14[[0, 2], :].T)

In [ ]:
# model_test = SpectralClustering(n_clusters=5, affinity='nearest_neighbors', assign_labels='kmeans')
model_test = OPTICS(min_samples=2)

In [ ]:
model = SpectralClustering(n_clusters=2, affinity='nearest_neighbors', assign_labels='kmeans')

In [ ]:
labels_km = estimator.labels_
labels_sc = model.fit_predict(data_z_L14[[0, 2], :].T)

In [ ]:
np.sum(~np.isfinite(data_z_L14[2, :]))

In [ ]:
labels_test = model_test.fit_predict(data_z_L14[[0, 2], :].T)

In [ ]:
labels_test

In [ ]:
np.shape(data_z_L14[[0, 2], :].T)

In [ ]:
data_z_L14[0, :]

In [ ]:
fig = plt.figure(figsize=(8,5))
ax1 = fig.add_subplot(1,1,1)

ax1.scatter(data_z_L14[0, :], data_z_L14[2, :], c=labels_km, edgecolor='k', s=50, cmap='inferno')
ax1.set_yscale('log')
ax1.set_ylim((1e23, 1e29))

In [ ]:
fig = plt.figure(figsize=(8,5))
ax1 = fig.add_subplot(1,1,1)

ax1.scatter(data_z_L14[0, :], data_z_L14[2, :], c='k', edgecolor='k', s=50, cmap='inferno')
ax1.set_yscale('log')
ax1.set_ylim((1e23, 1e29))

In [ ]:
fig = plt.figure(figsize=(8,5))
ax1 = fig.add_subplot(1,1,1)

ax1.scatter(large_sample_z, large_sample_L, c=labels_test, edgecolor='k', s=50, cmap='inferno')
ax1.set_yscale('log')